<a href="https://colab.research.google.com/github/SSPYX/Colab/blob/main/ExcelFormat_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy.core.multiarray import concatenate
import pandas as pd
import openpyxl
import numpy as np
import re
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

wb = openpyxl.load_workbook('/content/1010_SF_AR_Test.xlsx', data_only=True, rich_text=True)
indx_sheet = wb['Index']
add_col_idx = indx_sheet.max_column + 1
#sheet_names = wb.sheetnames
# Use the following SWITCH to define "scope" of the script. False = Do not apply, True = Apply
Font_Size_Switch = True
Row_Height_Switch = True
Border_Switch = True # False will not remove vertical borders
Column_Width_Switch = True
#print(wb.sheetnames)#
for title in wb.sheetnames:
  print("Start of", title)
  ws = wb[title]
#  hidden_flg = ws.sheet_state
  for name_range in wb.defined_names.values():
   char_idx = name_range.attr_text.find('!')
   #print(ws.sheet_state)
   if ws.title==name_range.attr_text[:char_idx] and ws.sheet_state == 'visible':
    #named_array = wb.defined_names[title]
    #print("I am processing",ws.title)
    cell_range=name_range.attr_text
    #print(cell_range)
    start_cell, end_cell = cell_range.split(':')
    sheet, start_cell = start_cell.split('!')
    start_cell = start_cell.replace("$","")
    end_cell = end_cell.replace("$","")
    start_row, start_col = openpyxl.utils.cell.coordinate_to_tuple(start_cell)
    end_row, end_col= openpyxl.utils.cell.coordinate_to_tuple(end_cell)
    row_height = 14.5
    index = 0
    num_count = 0
    cnt_nts = 0
    hidden_columns = []
    notes_exists = False
    min=0
    max=0
    h_min=0
    h_max=0
    true_end = end_col
    hiddenList=[]
    groupedList=[]

    #Thi block loops on rows defined in the named range to find Notes colum
    for row in ws.iter_rows():
        for row_cell in row:
         if row_cell.value =="Notes" or row_cell.value =="Note":
          notes_exists=True
          cnt_nts = row_cell.col_idx
          print("Notes Index position",cnt_nts)
          break
    # This block determines number of numeric columns

    for col_h in range(start_col , end_col + 1):
        sheet_cell = ws.cell(row=end_row, column=col_h)
        print("sheet scope", sheet_cell.value, col_h, bool )
        print(col_h, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden, ws.title )
        if col_h in groupedList: #col_h == true_end and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min:
          ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min = col_h
          ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max = col_h
          num_count += 1
          print("I am columns", col_h,"I am in condition 0")
          continue
        if col_h in hiddenList: #min and max and col_h > min and col_h <= max and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden :
             print("I am column", col_h, "and I am in condition 1")
             continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min == ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max: #This is a default scenario for visible columns
             #print(type(sheet_cell.value))
             if isinstance(sheet_cell.value, (int, float)): #(re.match(r'^#',last_row_values.number_format)) and col_h != cnt_nts: #
                  num_count += 1
                  print("I am column", col_h, "and I am in condition 2")
        elif ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max:   #This capture the first cell of hidden cell range
              hiddenList.clear()
              for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max + 1):
                            hiddenList.append(i)
              h_min = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min
              h_max = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max
              print("I am column", col_h, "and I am in condition 3", hiddenList )
              continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max:   #This captures any grouped columns that have min & max defined None and are visible in excel
              groupedList.clear()
              for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max + 1):
                            groupedList.append(i)
              min = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min
              max = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max
              num_count += 1
              print("I am column", col_h, "and I am in condition 4",min,max, groupedList)

    print("Numeric columns", num_count, ws.title)


    #Loop on rows defined in the named range
    for row in range(start_row, end_row + 1):
      index += 1
      if Row_Height_Switch is True:
        #set the row height to 14.5
        ws.row_dimensions[row].height=row_height
        Is_hidden = ws.row_dimensions[row].hidden
      groupedList.clear()
      hiddenList.clear()
      col_index=start_col + 1
# Loop on columns starting from the range defined in Named Range
      for col in range(start_col , end_col + 1):
       sheet_cell = ws.cell(row=row, column=col)
       if index==1: #Row 1
        last_row_values = ws.cell(row=end_row, column=col)
#******This block is to hide columns that come as visible due to excel functionality to Group columns that are hidden consecutively hidden********
        if col in groupedList: #col_h == true_end and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min:
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
        if col_h in hiddenList: #min and max and col_h > min and col_h <= max and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden :
          ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden = True
          continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min == ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max: #This is a default scenario for visible columns
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
        elif ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max:   #This capture the first cell of hidden cell range
          hiddenList.clear()
          for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max + 1):
                            hiddenList.append(i)
          print("I am column", col_h, "and I am in condition 3", hiddenList )
          continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max:   #This captures any grouped columns that have min & max defined None and are visible in excel
          groupedList.clear()
          for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max + 1):
                            groupedList.append(i)
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
          print("I am column", col_h, "and I am in condition 4",min,max, groupedList)
#********End of block*************

        if not Font_Size_Switch:
            font_size = sheet_cell.font.size
        else:
            # Set the font SIZE based on numeric columns
            if num_count <= 8:
              font_size = 9
            elif num_count in (9, 10):
              font_size = 8
            else:
              font_size = 7

 #******This block sets the Column Width at row of the defined range as this block is executed within Index 1 if condition above
        if Column_Width_Switch is True:
          if notes_exists and col==col_index:
              if num_count == 1:
                   col_width = 8.91
              elif num_count == 2:
                   col_width = 8.91
              elif num_count==3:
                   col_width = 8.91
              elif num_count==4:
                   col_width=8.18
              elif num_count==5:
                   col_width=7.82
              elif num_count==6:
                   col_width=8.91
              elif num_count==7:
                   col_width=8.55
              elif num_count==8:
                   col_width=8.18
              elif num_count==9:
                   col_width=7.82
              elif num_count==10:
                   col_width=7
              elif num_count==11:
                   col_width=6.18
              else:
                   col_width=6.18
              print("I am using", col_width, "for Notes columns in sheet", ws.title)
                # now update the worksheet with above column widths
              ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
          if col==start_col and not notes_exists: #This is for column 1
                #Set the default column WIDTH of first colum based on how many numeric columns
                if num_count == 1:
                    col_width = 65.55 + 8.91
                elif num_count == 2:
                    col_width = 53.55 + 8.91
                elif num_count==3:
                    col_width = 41.45 + 8.91
                elif num_count==4:
                    col_width=34.18 + 8.18
                elif num_count==5:
                    col_width=26.09 + 7.82
                elif num_count==6:
                    col_width=49 + 8.91
                elif num_count==7:
                    col_width=40.82 + 8.55
                elif num_count==8:
                    col_width=33.64 + 8.18
                elif num_count==9:
                    col_width=27.55 + 7.82
                elif num_count==10:
                    col_width=27.91 + 7
                elif num_count==11:
                    col_width=61.4 + 6.18
                else:
                    col_width=21.64 + 6.18
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
                # set the column widths in worksheet
                print(col, col_width, "for label")
          if col==start_col and notes_exists: #This is for column 1
                #Set the default column WIDTH of first colum based on how many numeric columns
                if num_count == 1:
                    col_width = 65.55
                elif num_count == 2:
                    col_width = 53.55
                elif num_count==3:
                    col_width = 41.45
                elif num_count==4:
                    col_width=34.18
                elif num_count==5:
                    col_width=26.09
                elif num_count==6:
                    col_width=49
                elif num_count==7:
                    col_width=40.82
                elif num_count==8:
                    col_width=33.64
                elif num_count==9:
                    col_width=27.55
                elif num_count==10:
                    col_width=27.91
                elif num_count==11:
                    col_width=61.4
                else:
                    col_width=21.64
                # set the column widths in worksheet
                print(col, col_width, "for label")
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
          if isinstance(last_row_values.value,(int, float)) and col!=cnt_nts and col!=start_col: #(re.match(r'^#',last_row_values.number_format)) and col!=cnt_nts:
                #Set the default column WIDTH for numeric column based on how many amount columns are in sheet
                if num_count <= 3 or num_count==6:
                   col_width = 11.45
                elif num_count == 4:
                   col_width = 10.36
                elif num_count==5:
                   col_width = 9.91
                elif num_count==7:
                   col_width=10.91
                elif num_count==8:
                   col_width=10.36
                elif num_count==9:
                   col_width=9.64
                elif num_count==10:
                   col_width=8.82
                else:
                   col_width=7.91
                print("I am using", col_width, "for amount")
                # set the column widths in worksheet
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
        elif not Column_Width_Switch:
          ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width
#********This block sets the font size and formatting properties of cell***************
       font_prpts = {}
       if Font_Size_Switch and index == 1:
         #set the font properties for row 1
         fstyle_bold = sheet_cell.font.bold
         fstyle_italic = sheet_cell.font.italic
         fstyle_size = sheet_cell.font.size
         #print("I am setting", fstyle_bold, "font properties of", row, col)
         font_prpts= { "name": "Arial",
              "size": fstyle_size,
              "bold": fstyle_bold,
              "italic": fstyle_italic,
              "vertAlign":"baseline",
              "underline":"none",
              "strike":False }
         font = Font(**font_prpts)
         sheet_cell.font = font
       elif Font_Size_Switch and index != 1:
         fstyle_bold = sheet_cell.font.bold
         fstyle_italic = sheet_cell.font.italic
         fstyle_size = sheet_cell.font.size
         #print("I am setting", fstyle_bold, "font properties of", row, col)
        #set the font properties for remaining rows
         font_prpts= { "name": "Arial",
              "size": fstyle_size if not Font_Size_Switch else 9.0,
              "bold": fstyle_bold,
              "italic": fstyle_italic,
              "vertAlign":"baseline",
              "underline":"none",
              "strike":False }
       font = Font(**font_prpts)
       sheet_cell.font = font
#*********This block sets the formatting properties*****************#
       if Border_Switch == True and not 'PPE' in ws.title:
          if sheet_cell.border.left.border_style == "thin" or sheet_cell.border.right.border_style == "thin":
             sheet_cell.border.left.border_style = 'none'
             sheet_cell.border.right.border_style = 'none'
             print("a border is removed at cell location", col, ws.title)
       if row == start_row and sheet_cell.value=="Notes" or sheet_cell.value=="Note":
             sheet_cell.alignment = Alignment(horizontal='right',
                                              vertical = 'bottom',
                                              wrap_text=True)
       elif row == start_row and sheet_cell.value!="Notes" or sheet_cell.value=="Note":
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=True)
       elif row != start_row and col == start_col:
             cell_index = sheet_cell.alignment.indent
             sheet_cell.alignment = Alignment(horizontal='left',
                                                vertical = 'bottom',
                                                wrap_text=True,
                                                indent=cell_index)
       elif col==col_index and notes_exists:
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=True)
               sheet_cell.number_format = 'General'

       else:
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=False)

  for irow in indx_sheet.iter_rows(min_row=2, max_row=indx_sheet.max_row, min_col=1, max_col=indx_sheet.max_column):
    for icell in irow:
      if icell.value==ws.title:
        indx_sheet.cell(row=icell.row, column=add_col_idx, value=num_count)

wb.save('test.xlsx')

In [ ]:
#**************************************************************************************This is a version that retains the formulas**************************************************************************
from numpy.core.multiarray import concatenate
import pandas as pd
import openpyxl
import numpy as np
import re
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

wb = openpyxl.load_workbook('/content/1010_SF_AR_Test.xlsx', data_only=False, rich_text=True)
indx_sheet = wb['Index']
add_col_idx = indx_sheet.max_column + 1
#sheet_names = wb.sheetnames
# Use the following SWITCH to define "scope" of the script. False = Do not apply, True = Apply
Font_Size_Switch = True
Row_Height_Switch = True
Border_Switch = True # False will not remove vertical borders
Column_Width_Switch = True
#print(wb.sheetnames)#
for title in wb.sheetnames:
  print("Start of", title)
  ws = wb[title]
#  hidden_flg = ws.sheet_state
  for name_range in wb.defined_names.values():
   char_idx = name_range.attr_text.find('!')
   #print(ws.sheet_state)
   if ws.title==name_range.attr_text[:char_idx] and ws.sheet_state == 'visible':
    #named_array = wb.defined_names[title]
    #print("I am processing",ws.title)
    cell_range=name_range.attr_text
    #print(cell_range)
    start_cell, end_cell = cell_range.split(':')
    sheet, start_cell = start_cell.split('!')
    start_cell = start_cell.replace("$","")
    end_cell = end_cell.replace("$","")
    start_row, start_col = openpyxl.utils.cell.coordinate_to_tuple(start_cell)
    end_row, end_col= openpyxl.utils.cell.coordinate_to_tuple(end_cell)
    row_height = 14.5
    index = 0
    num_count = 0
    cnt_nts = 0
    hidden_columns = []
    notes_exists = False
    min=0
    max=0
    h_min=0
    h_max=0
    true_end = end_col
    hiddenList=[]
    groupedList=[]

    #Thi block loops on rows defined in the named range to find Notes colum
    for row in ws.iter_rows():
        for row_cell in row:
         if row_cell.value =="Notes" or row_cell.value =="Note":
          notes_exists=True
          cnt_nts = row_cell.col_idx
          print("Notes Index position",cnt_nts)
          break
    # This block determines number of numeric columns

    for col_h in range(start_col , end_col + 1):
        sheet_cell = ws.cell(row=end_row, column=col_h)
        #print("sheet scope", sheet_cell.value, col_h, bool )
        #print(col_h, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden, ws.title )
        if col_h in groupedList: #col_h == true_end and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min:
          ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min = col_h
          ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max = col_h
          num_count += 1
          print("I am columns", col_h,"I am in condition 0")
          continue
        if col_h in hiddenList: #min and max and col_h > min and col_h <= max and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden :
             print("I am column", col_h, "and I am in condition 1")
             continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min == ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max: #This is a default scenario for visible columns
             #print(type(sheet_cell.value))
             if (re.match(r'^#',last_row_values.number_format)) and col_h != cnt_nts and col_h !=start_col : # isinstance(sheet_cell.value, (int, float)): #
                  num_count += 1
                  print("I am column", col_h, "and I am in condition 2")
        elif ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max:   #This capture the first cell of hidden cell range
              hiddenList.clear()
              for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max + 1):
                            hiddenList.append(i)
              h_min = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min
              h_max = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max
              print("I am column", col_h, "and I am in condition 3", hiddenList )
              continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max:   #This captures any grouped columns that have min & max defined None and are visible in excel
              groupedList.clear()
              for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max + 1):
                            groupedList.append(i)
              min = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min
              max = ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].max
              num_count += 1
              print("I am column", col_h, "and I am in condition 4",min,max, groupedList)

    print("Numeric columns", num_count, ws.title)


    #Loop on rows defined in the named range
    for row in range(start_row, end_row + 1):
      index += 1
      if Row_Height_Switch is True:
        #set the row height to 14.5
        ws.row_dimensions[row].height=row_height
        Is_hidden = ws.row_dimensions[row].hidden
      groupedList.clear()
      hiddenList.clear()
      col_index=start_col + 1
# Loop on columns starting from the range defined in Named Range
      for col in range(start_col , end_col + 1):
       sheet_cell = ws.cell(row=row, column=col)
       if index==1: #Row 1
        last_row_values = ws.cell(row=end_row, column=col)
#******This block is to hide columns that come as visible due to excel functionality to Group columns that are hidden consecutively hidden********
        if col in groupedList: #col_h == true_end and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].min:
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
        if col_h in hiddenList: #min and max and col_h > min and col_h <= max and not ws.column_dimensions[openpyxl.utils.get_column_letter(col_h)].hidden :
          ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden = True
          continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min == ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max: #This is a default scenario for visible columns
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
        elif ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max:   #This capture the first cell of hidden cell range
          hiddenList.clear()
          for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max + 1):
                            hiddenList.append(i)
          print("I am column", col_h, "and I am in condition 3", hiddenList )
          continue
        elif not ws.column_dimensions[openpyxl.utils.get_column_letter(col)].hidden and ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min != ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max:   #This captures any grouped columns that have min & max defined None and are visible in excel
          groupedList.clear()
          for i in range(ws.column_dimensions[openpyxl.utils.get_column_letter(col)].min, ws.column_dimensions[openpyxl.utils.get_column_letter(col)].max + 1):
                            groupedList.append(i)
          fstyle_bold = sheet_cell.font.bold
          fstyle_italic = sheet_cell.font.italic
          print("I am column", col_h, "and I am in condition 4",min,max, groupedList)
#********End of block*************

        if not Font_Size_Switch:
            font_size = sheet_cell.font.size
        else:
            # Set the font SIZE based on numeric columns
            if num_count <= 8:
              font_size = 9
            elif num_count in (9, 10):
              font_size = 8
            else:
              font_size = 7

 #******This block sets the Column Width at row of the defined range as this block is executed within Index 1 if condition above
        if Column_Width_Switch is True:
          if notes_exists and col==col_index:
              if num_count == 1:
                   col_width = 8.91
              elif num_count == 2:
                   col_width = 8.91
              elif num_count==3:
                   col_width = 8.91
              elif num_count==4:
                   col_width=8.18
              elif num_count==5:
                   col_width=7.82
              elif num_count==6:
                   col_width=8.91
              elif num_count==7:
                   col_width=8.55
              elif num_count==8:
                   col_width=8.18
              elif num_count==9:
                   col_width=7.82
              elif num_count==10:
                   col_width=7
              elif num_count==11:
                   col_width=6.18
              else:
                   col_width=6.18
              print("I am using", col_width, "for Notes columns in sheet", ws.title)
                # now update the worksheet with above column widths
              ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
          if col==start_col and not notes_exists: #This is for column 1
                #Set the default column WIDTH of first colum based on how many numeric columns
                if num_count == 1:
                    col_width = 65.55 + 8.91
                elif num_count == 2:
                    col_width = 53.55 + 8.91
                elif num_count==3:
                    col_width = 41.45 + 8.91
                elif num_count==4:
                    col_width=34.18 + 8.18
                elif num_count==5:
                    col_width=26.09 + 7.82
                elif num_count==6:
                    col_width=49 + 8.91
                elif num_count==7:
                    col_width=40.82 + 8.55
                elif num_count==8:
                    col_width=33.64 + 8.18
                elif num_count==9:
                    col_width=27.55 + 7.82
                elif num_count==10:
                    col_width=27.91 + 7
                elif num_count==11:
                    col_width=61.4 + 6.18
                else:
                    col_width=21.64 + 6.18
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
                # set the column widths in worksheet
                print(col, col_width, "for label")
          if col==start_col and notes_exists: #This is for column 1
                #Set the default column WIDTH of first colum based on how many numeric columns
                if num_count == 1:
                    col_width = 65.55
                elif num_count == 2:
                    col_width = 53.55
                elif num_count==3:
                    col_width = 41.45
                elif num_count==4:
                    col_width=34.18
                elif num_count==5:
                    col_width=26.09
                elif num_count==6:
                    col_width=49
                elif num_count==7:
                    col_width=40.82
                elif num_count==8:
                    col_width=33.64
                elif num_count==9:
                    col_width=27.55
                elif num_count==10:
                    col_width=27.91
                elif num_count==11:
                    col_width=61.4
                else:
                    col_width=21.64
                # set the column widths in worksheet
                print(col, col_width, "for label")
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
          if (re.match(r'^#',last_row_values.number_format)) and col!=cnt_nts and col!=start_col: #isinstance(last_row_values, (int, float)): #
                #Set the default column WIDTH for numeric column based on how many amount columns are in sheet
                if num_count <= 3 or num_count==6:
                   col_width = 11.45
                elif num_count == 4:
                   col_width = 10.36
                elif num_count==5:
                   col_width = 9.91
                elif num_count==7:
                   col_width=10.91
                elif num_count==8:
                   col_width=10.36
                elif num_count==9:
                   col_width=9.64
                elif num_count==10:
                   col_width=8.82
                else:
                   col_width=7.91
                print("I am using", col_width, "for amount")
                # set the column widths in worksheet
                ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=col_width
        elif not Column_Width_Switch:
          ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width=ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width
#********This block sets the font size and formatting properties of cell***************
       font_prpts = {}
       if Font_Size_Switch and index == 1:
         #set the font properties for row 1
         fstyle_bold = sheet_cell.font.bold
         fstyle_italic = sheet_cell.font.italic
         fstyle_size = sheet_cell.font.size
         #print("I am setting", fstyle_bold, "font properties of", row, col)
         font_prpts= { "name": "Arial",
              "size": fstyle_size,
              "bold": fstyle_bold,
              "italic": fstyle_italic,
              "vertAlign":"baseline",
              "underline":"none",
              "strike":False }
         font = Font(**font_prpts)
         sheet_cell.font = font
       elif Font_Size_Switch and index != 1:
         fstyle_bold = sheet_cell.font.bold
         fstyle_italic = sheet_cell.font.italic
         fstyle_size = sheet_cell.font.size
         #print("I am setting", fstyle_bold, "font properties of", row, col)
        #set the font properties for remaining rows
         font_prpts= { "name": "Arial",
              "size": fstyle_size if not Font_Size_Switch else 9.0,
              "bold": fstyle_bold,
              "italic": fstyle_italic,
              "vertAlign":"baseline",
              "underline":"none",
              "strike":False }
       font = Font(**font_prpts)
       sheet_cell.font = font
#*********This block sets the formatting properties*****************#
       if Border_Switch == True and not 'PPE' in ws.title:
          if sheet_cell.border.left.border_style == "thin" or sheet_cell.border.right.border_style == "thin":
             sheet_cell.border.left.border_style = 'none'
             sheet_cell.border.right.border_style = 'none'
             print("a border is removed at cell location", col, ws.title)
       if row == start_row and sheet_cell.value=="Notes" or sheet_cell.value=="Note":
             sheet_cell.alignment = Alignment(horizontal='right',
                                              vertical = 'bottom',
                                              wrap_text=True)
       elif row == start_row and sheet_cell.value!="Notes" or sheet_cell.value=="Note":
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=True)
       elif row != start_row and col == start_col:
             cell_index = sheet_cell.alignment.indent
             sheet_cell.alignment = Alignment(horizontal='left',
                                                vertical = 'bottom',
                                                wrap_text=True,
                                                indent=cell_index)
       elif col==col_index and notes_exists:
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=True)
               sheet_cell.number_format = 'General'

       else:
               sheet_cell.alignment = Alignment(horizontal='right',
                                                vertical = 'bottom',
                                                wrap_text=False)

  for irow in indx_sheet.iter_rows(min_row=2, max_row=indx_sheet.max_row, min_col=1, max_col=indx_sheet.max_column):
    for icell in irow:
      if icell.value==ws.title:
        indx_sheet.cell(row=icell.row, column=add_col_idx, value=num_count)

wb.save('test.xlsx')

KeyboardInterrupt: ignored